In [1]:
import pandas as pd
import numpy as np

In [2]:
from lxml import html
import requests

In [3]:
main_list = []
list_of_names = []

for i in range(1,251):
    page = requests.get('https://kolesa.kz/cars/?page=' + str(i))
    tree = html.fromstring(page.content)
    
    s1 = tree.xpath('//span[@class="a-el-info-title"]/a/@href')
    s2 = tree.xpath('//a[@class="list-link ddl_product_link"]/text()')
    for j in s1:
        main_list.append(j)
    for j in s2:
        list_of_names.append(j)
    if i%50==0:
        print(i)

50
100
150
200
250


In [4]:
main_list[:5]

['/a/show/43865599',
 '/a/show/40595130',
 '/a/show/47049673',
 '/a/show/85834690',
 '/a/show/86238597']

In [5]:
list_of_names[:5]

['Toyota Camry',
 'Mercedes-Benz G 63 AMG',
 'Toyota Corolla',
 'Mercedes-Benz E 220',
 'Mitsubishi Outlander']

In [6]:
print(len(main_list))
print(len(list_of_names))

5000
5000


In [7]:
main_list2 = ['https://kolesa.kz' + i for i in main_list]
main_list2[4999]

'https://kolesa.kz/a/show/86428411'

In [8]:
df = {'links': main_list2, 'names': list_of_names}
df = pd.DataFrame(df)
df.head()

,links,names
0,https://kolesa.kz/a/show/43865599,Toyota Camry
1,https://kolesa.kz/a/show/40595130,Mercedes-Benz G 63 AMG
2,https://kolesa.kz/a/show/47049673,Toyota Corolla
3,https://kolesa.kz/a/show/85834690,Mercedes-Benz E 220
4,https://kolesa.kz/a/show/86238597,Mitsubishi Outlander


In [20]:
df['links'][0]

'https://kolesa.kz/a/show/43865599'

In [37]:
dictionary_list = []
d = []
false_hrefs = []
counter = 1
for i in range(len(df['links'])):
    page = requests.get(df['links'][i])
    tree = html.fromstring(page.content)
    
    s1 = tree.xpath('//dt[@class="value-title"]/span/text()')
    s2 = tree.xpath('//dd[@class="value"]/text()')
    
    dictionary = {}
    
    if len(s1) == len(s2):
        for j in range(len(s1)):        
            dictionary[s1[j]] = s2[j].strip()
    else:
        false_hrefs.append('https://kolesa.kz/' + i)
        
    dictionary_list.append(dictionary)
    if counter%500==0:
        print(counter)
    counter += 1


500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [38]:
print(len(dictionary_list))
print(dictionary_list[2000:2002])

5000
[{'Кузов': 'седан', 'Цвет': 'синий металлик', 'Город': 'Уральск', 'Объем двигателя, л': '1.6 (бензин)', 'Коробка передач': 'механика', 'Руль': 'слева', 'Привод': 'передний привод', 'Растаможен': 'Да', 'Пробег': '90 500 км'}, {'Кузов': 'седан', 'Цвет': 'золотистый металлик', 'Город': 'Жаркент', 'Объем двигателя, л': '3 (бензин)', 'Коробка передач': 'автомат', 'Руль': 'слева', 'Растаможен': 'Да'}]


In [47]:
page = requests.get(df['links'][0])
tree = html.fromstring(page.content)

s1 = tree.xpath('//div[@class="offer__price"]/text()')

s2 = tree.xpath('//span[@class="year"]/text()')

print(s2[0])

 2018


In [49]:
list_of_prices = []
list_of_year = []

c = 1

for i in df['links']:
    page = requests.get(i)
    tree = html.fromstring(page.content)
    
    s1 = tree.xpath('//div[@class="offer__price"]/text()')
    s2 = tree.xpath('//span[@class="year"]/text()')
    
    if len(s1)==0:
        list_of_prices.append('')
    else:
        list_of_prices.append(s1[0].strip())
    
    if len(s2)==0:
        list_of_year.append('')
    else:
        list_of_year.append(s2[0].strip())
        
    if c%500==0:
        print(c)
    c+=1

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [50]:
print(len(list_of_prices))
print(len(list_of_year))

5000
5000


In [51]:
set_of_keys = []
for i in dictionary_list:
    for j in i.keys():
        set_of_keys.append(j)

set_of_keys = set(set_of_keys)
set_of_keys

{'VIN',
 'Город',
 'Коробка передач',
 'Кузов',
 'Объем двигателя, л',
 'Привод',
 'Пробег',
 'Растаможен',
 'Руль',
 'Цвет'}

In [52]:
len(dictionary_list[0].keys())

9

In [53]:
len(dictionary_list)

5000

In [54]:
counter = 0
for i in range(len(dictionary_list)):
    if len(dictionary_list[i].keys()) == 0:
        counter +=1
counter

2

In [55]:
list_of_names = list_of_names[:5000]
for i in range(len(dictionary_list)-1,-1,-1):
    if len(dictionary_list[i].keys()) == 0:
        list_of_names.pop(i)
        list_of_year.pop(i)
        list_of_prices.pop(i)

In [56]:
print(len(list_of_names))
print(len(list_of_year))
print(len(list_of_prices))

4998
4998
4998


In [57]:
5000-2

4998

In [58]:
for i in range(4999,-1,-1):
    if len(dictionary_list[i].keys()) == 0:
        dictionary_list.pop(i)

In [59]:
len(dictionary_list)

4998

In [60]:
for i in range(len(list(set_of_keys))):
    print(list(set_of_keys)[i])

Кузов
Цвет
Город
Руль
Растаможен
Коробка передач
VIN
Пробег
Объем двигателя, л
Привод


In [61]:
list1_from_dictionary = []
list2_from_dictionary = []
list3_from_dictionary = []
list4_from_dictionary = []
list5_from_dictionary = []
list6_from_dictionary = []
list7_from_dictionary = []
list8_from_dictionary = []
list9_from_dictionary = []
list10_from_dictionary = []

for i in range(len(dictionary_list)):
    if list(set_of_keys)[0] in dictionary_list[i].keys():
        list1_from_dictionary.append(dictionary_list[i][list(set_of_keys)[0]])
    else:
        list1_from_dictionary.append('')
    if list(set_of_keys)[1] in dictionary_list[i].keys():
        list2_from_dictionary.append(dictionary_list[i][list(set_of_keys)[1]])
    else:
        list2_from_dictionary.append('')
    if list(set_of_keys)[2] in dictionary_list[i].keys():
        list3_from_dictionary.append(dictionary_list[i][list(set_of_keys)[2]])
    else:
        list3_from_dictionary.append('')
    if list(set_of_keys)[3] in dictionary_list[i].keys():
        list4_from_dictionary.append(dictionary_list[i][list(set_of_keys)[3]])
    else:
        list4_from_dictionary.append('')
    if list(set_of_keys)[4] in dictionary_list[i].keys():
        list5_from_dictionary.append(dictionary_list[i][list(set_of_keys)[4]])
    else:
        list5_from_dictionary.append('')
    if list(set_of_keys)[5] in dictionary_list[i].keys():
        list6_from_dictionary.append(dictionary_list[i][list(set_of_keys)[5]])
    else:
        list6_from_dictionary.append('')
    if list(set_of_keys)[6] in dictionary_list[i].keys():
        list7_from_dictionary.append(dictionary_list[i][list(set_of_keys)[6]])
    else:
        list7_from_dictionary.append('')
    if list(set_of_keys)[7] in dictionary_list[i].keys():
        list8_from_dictionary.append(dictionary_list[i][list(set_of_keys)[7]])
    else:
        list8_from_dictionary.append('')
    if list(set_of_keys)[8] in dictionary_list[i].keys():
        list9_from_dictionary.append(dictionary_list[i][list(set_of_keys)[8]])
    else:
        list9_from_dictionary.append('')
    if list(set_of_keys)[9] in dictionary_list[i].keys():
        list10_from_dictionary.append(dictionary_list[i][list(set_of_keys)[9]])
    else:
        list10_from_dictionary.append('')

In [62]:
print(len(list1_from_dictionary))
print(len(list2_from_dictionary))
print(len(list3_from_dictionary))
print(len(list4_from_dictionary))
print(len(list5_from_dictionary))
print(len(list6_from_dictionary))
print(len(list7_from_dictionary))
print(len(list8_from_dictionary))
print(len(list9_from_dictionary))
print(len(list10_from_dictionary))

4998
4998
4998
4998
4998
4998
4998
4998
4998
4998


In [63]:
list_from_dictionary = [list1_from_dictionary,
                        list2_from_dictionary,
                        list3_from_dictionary,
                        list4_from_dictionary,
                        list5_from_dictionary,
                        list6_from_dictionary,
                        list7_from_dictionary,
                        list8_from_dictionary,
                        list9_from_dictionary,
                        list10_from_dictionary, 
                        list_of_year,
                        list_of_prices]

In [64]:
dictionary_big = {}
for i in range(len(list(set_of_keys))):
    dictionary_big[list(set_of_keys)[i]] = list_from_dictionary[i]

In [65]:
dictionary_big['Year'] = list_from_dictionary[10]
dictionary_big['Price'] = list_from_dictionary[11]

In [66]:
df = pd.DataFrame.from_dict(dictionary_big)
df.head()

,Price,VIN,Year,Город,Коробка передач,Кузов,"Объем двигателя, л",Привод,Пробег,Растаможен,Руль,Цвет
0,13 800 000,,2018,Алматы,автомат,седан,2.5 (бензин),передний привод,1 км,Да,слева,серебристый металлик
1,68 000 000,,2014,Алматы,автомат,внедорожник,5.5 (бензин),,10 300 км,Да,слева,черный
2,5 499 999,,2013,Алматы,автомат,седан,1.6 (бензин),передний привод,39 258 км,Да,слева,серый металлик
3,1 850 000,,1993,Шымкент,автомат,седан,2.2 (газ-бензин),задний привод,,Да,слева,серый металлик
4,6 100 000,,2013,Актобе,автомат,кроссовер,2.4 (бензин),полный привод,60 000 км,Да,слева,серый


In [67]:
len(df)

4998

In [68]:
df = df.drop(columns=['VIN'], axis=1)

In [69]:
df.head()

,Price,Year,Город,Коробка передач,Кузов,"Объем двигателя, л",Привод,Пробег,Растаможен,Руль,Цвет
0,13 800 000,2018,Алматы,автомат,седан,2.5 (бензин),передний привод,1 км,Да,слева,серебристый металлик
1,68 000 000,2014,Алматы,автомат,внедорожник,5.5 (бензин),,10 300 км,Да,слева,черный
2,5 499 999,2013,Алматы,автомат,седан,1.6 (бензин),передний привод,39 258 км,Да,слева,серый металлик
3,1 850 000,1993,Шымкент,автомат,седан,2.2 (газ-бензин),задний привод,,Да,слева,серый металлик
4,6 100 000,2013,Актобе,автомат,кроссовер,2.4 (бензин),полный привод,60 000 км,Да,слева,серый


In [70]:
dictionary_big['Names'] = list_of_names

In [71]:
df['Names'] = list_of_names

In [72]:
df.head()

,Price,Year,Город,Коробка передач,Кузов,"Объем двигателя, л",Привод,Пробег,Растаможен,Руль,Цвет,Names
0,13 800 000,2018,Алматы,автомат,седан,2.5 (бензин),передний привод,1 км,Да,слева,серебристый металлик,Toyota Camry
1,68 000 000,2014,Алматы,автомат,внедорожник,5.5 (бензин),,10 300 км,Да,слева,черный,Mercedes-Benz G 63 AMG
2,5 499 999,2013,Алматы,автомат,седан,1.6 (бензин),передний привод,39 258 км,Да,слева,серый металлик,Toyota Corolla
3,1 850 000,1993,Шымкент,автомат,седан,2.2 (газ-бензин),задний привод,,Да,слева,серый металлик,Mercedes-Benz E 220
4,6 100 000,2013,Актобе,автомат,кроссовер,2.4 (бензин),полный привод,60 000 км,Да,слева,серый,Mitsubishi Outlander


In [73]:
df.to_csv('datas.csv')